## Millikan Oil Drop Experiment

In [ ]:
import math
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt

Assume that we have made the following measurements of the charge on the oil drops. These are all in units of 1e-19 coulombs.

In [ ]:
measurements = np.array([1.46, 3.18, 4.86, 3.26, 6.48, 1.52, 3.10, 3.26, 4.87, 1.60])

We can build a probablistic model to describe this noisy measurement process.

In [ ]:
N = len(measurements)

model = pm.Model()

with model:
    
    charge = pm.Uniform("charge", 1.0, 2.0)

    number = pm.Poisson('number', 1.0, shape=(N))
        
    mu = (1 + number) * charge
    
    sd = pm.Exponential("sd", 1.0 / 0.1)

    observation = pm.Gamma("obs", mu=mu, sigma=sd, observed=measurements) 

We run the model with some default settings.

In [ ]:
with model:
    
    idata = pm.sample(2000)

We can quickly review the trace. In this case we need to select the random variables because *traceplot* does not know how to draw the discrete *number* random variable.

In [ ]:
pm.plot_trace(idata, var_names=["charge", "sd"], figsize=(8, 8));

In [ ]:
pm.summary(idata, var_names=["charge", "sd"])

Finally when we are happy we can generate some final plots and report on the results.

In [ ]:
charge = np.asarray(idata.posterior.get('charge')[0])
sd = np.asarray(idata.posterior.get('sd')[0])

fig, axes = plt.subplots(2, 1, figsize=(6, 6))

axes[0].hist(charge, histtype='stepfilled', bins=100, alpha=0.85, color="#467821", density=True)
axes[0].set_xlim(1, 2)
axes[0].set_title('Charge On The Electron ($e$)')
axes[0].set_xlabel('Coulombs (x $10^{-19}$ C)')

axes[1].hist(sd, histtype='stepfilled', bins=100, alpha=0.85, color="#A60628", density=True)
axes[1].set_xlim(0, 1)
axes[1].set_title('Measurement Error ($\sigma$)')
axes[1].set_xlabel('Coulombs (x $10^{-19}$ C)')

plt.tight_layout()

plt.savefig('plot.pdf')

We can also explore the mean number of charges on each oil drop.

In [ ]:
x = np.linspace(1, N, N)

number = 1 + np.mean(np.asarray(idata.posterior.get('number')[0]), 0)

plt.figure(figsize=(6, 4))
plt.bar(x=x, height=number)

plt.xticks(x)
plt.title("Expected Number of Electrons on Oil Drop")
plt.xlabel("ID of Measurement")
plt.ylim(0, 5);